In [1]:
##涨跌停计算器

##### 参数：价格 ######
currentPrice = 11.32  
###################

def calculateLimitPrice(price):
    lowRate = 0.9
    highRate = 1.1
    smallNum = 0.00001
    low = round(price*lowRate + smallNum, 2)
    high = round(price*highRate + smallNum, 2)
    return [low, high]

print(calculateLimitPrice(currentPrice))

[10.19, 12.45]


In [87]:
###每日涨停（不考虑ST垃圾股）
import tushare as ts
import numpy as np
from pandas import DataFrame

try:
    settingFile = open('/data/stock-setting.txt', 'r')
    settings = settingFile.read()
    token = settings.split()[0]
    secretUrl = settings.split()[1]
    settingFile.close()
except IOError:
    print('File is not accessible.')
pro = ts.pro_api(token)

dailyInfo = pro.daily(trade_date = '20190327', fields = 'ts_code, close, pre_close')
shape = dailyInfo.shape

def calculateLimitPrice(price):
    lowRate = 0.9
    highRate = 1.1
    smallNum = 0.00001
    low = round(price*lowRate + smallNum, 2)
    high = round(price*highRate + smallNum, 2)
    return [low, high]

def isLimitHighPrice(close, pre_close):
    limitHighPrice = calculateLimitPrice(pre_close)
    if limitHighPrice[1] == close:
        return True
    else:
        return False

    stocks = DataFrame([], columns=['code','name'])
for i in range(0, shape[0]):    
    if isLimitHighPrice(dailyInfo.iloc[i, 1], dailyInfo.iloc[i, 2]):
        code = dailyInfo.iloc[i, 0]
        ###获取股票名称
        nameHistory = pro.namechange(ts_code = code, fields='name')
        name = nameHistory.iloc[0, 0]
        stocks = stocks.append(DataFrame({'code':code,'name':name}, index = [code]))
print(stocks)

                code  name
66         000159.SZ  国际实业
144        000558.SZ  莱茵体育
166        000590.SZ  启迪古汉
182        000610.SZ  西安旅游
206        000639.SZ  西王食品
211        000656.SZ  金科股份
227        000677.SZ  恒天海龙
262        000721.SZ  西安饮食
264        000723.SZ  美锦能源
277        000739.SZ  普洛药业
290        000766.SZ  通化金马
317        000806.SZ  ST银河
321        000811.SZ  冰轮环境
323        000813.SZ  德展健康
444        000990.SZ  诚志股份
495        002040.SZ   南京港
570        002118.SZ  紫鑫药业
623        002172.SZ  澳洋健康
642        002191.SZ  劲嘉股份
676        002227.SZ   奥特迅
698        002249.SZ  大洋电机
748        002300.SZ  太阳电缆
837        002390.SZ  信邦制药
887        002440.SZ  闰土股份
960        002513.SZ  蓝丰生化
978        002532.SZ  新界泵业
1011       002565.SZ  顺灏股份
1053       002607.SZ  中公教育
1080       002636.SZ  金安国纪
1114       002670.SZ  国盛金控
...              ...   ...
600352.SH  600352.SH  浙江龙盛
600356.SH  600356.SH  恒丰纸业
600460.SH  600460.SH   士兰微
600479.SH  600479.SH  千金药业
600493.SH  600493.SH  凤竹纺织
6

In [97]:
# print(stocks.loc['601008.SH'][1])

# stocks.index = stocks.name
# print(stocks.loc['莱茵体育'])

print(0.11/3.43)

0.03206997084548105


In [1]:
###涨停跟踪
import tushare as ts
import numpy as np
from pandas import DataFrame
from pyecharts import Bar

try:
    settingFile = open('/data/stock-setting.txt', 'r')
    settings = settingFile.read()
    token = settings.split()[0]
    secretUrl = settings.split()[1]
    settingFile.close()
except IOError:
    print('File is not accessible.')
pro = ts.pro_api(token)

date = ['20190320', '20190321', '20190322', '20190325', '20190326', '20190327', '20190328', '20190329']

################函数############################
def calculateLimitPrice(price):
    lowRate = 0.9
    highRate = 1.1
    smallNum = 0.00001
    low = round(price*lowRate + smallNum, 2)
    high = round(price*highRate + smallNum, 2)
    return [low, high]

def isLimitHighPrice(close, pre_close):
    limitHighPrice = calculateLimitPrice(pre_close)
    if limitHighPrice[1] == close:
        return True
    else:
        return False
def belongTo(close, pre_close):
    var = (close - pre_close) / pre_close
    if isLimitHighPrice(close, pre_close):
        return 6
    elif var >= 0.07:
        return 5
    elif var >= 0.03:
        return 4
    elif var >= 0:
        return 3
    elif var > -0.03:
        return 2
    elif var >-0.07:
        return 1
    else:
        return 0    
#################################################
resultPercent = []
twoDayLimitUpStocks = []
for i in range(0, len(date)-1):
    today = date[i]
    nextDay = date[i+1]
    dailyInfo = pro.daily(trade_date = today, fields = 'ts_code, close, pre_close')
    nextDayInfo = pro.daily(trade_date = nextDay, fields = 'ts_code, close, pre_close')
    
    nextDayInfo.index = nextDayInfo.ts_code
    stocks = []
    percent = [0, 0, 0, 0, 0, 0, 0]
    for j in range(0, dailyInfo.shape[0]):    
        if isLimitHighPrice(dailyInfo.iloc[j, 1], dailyInfo.iloc[j, 2]):
            code = dailyInfo.iloc[j, 0]
            stocks.append(code)
    
    for j in range(0, len(stocks)):
        code = stocks[j]
        if code in nextDayInfo.index:
            info = nextDayInfo.loc[code]
            close = info[1]
            pre_close = info[2]
            catagory = belongTo(close, pre_close)
            percent[catagory] = percent[catagory] + 1
            if i == len(date) - 2 and catagory == 6:
                twoDayLimitUpStocks.append(code)
    resultPercent.append(percent)
print('今天连板股票共' + str(len(twoDayLimitUpStocks)) + '个')
print(twoDayLimitUpStocks)    
    
bar = Bar("涨停隔日分布图")
xLabel = date[0:len(date)-1]
catagory = ['[-10%, -7%]', '(-7%, -3%]', '(-3%, 0%)', '[0%, 3%)', '[3%, 7%)', '[7%, 10%)', '连板']
for i in range(0, len(catagory)):
    count = []
    for percent in resultPercent:
        count.append(percent[i])
    bar.add(catagory[i], xLabel, count, is_stack=True, legend_top = 'bottom')
bar


今天连板股票共20个
['002177.SZ', '300267.SZ', '300527.SZ', '300411.SZ', '002958.SZ', '603681.SH', '600146.SH', '603110.SH', '300766.SZ', '300762.SZ', '002761.SZ', '300110.SZ', '000927.SZ', '000590.SZ', '002265.SZ', '002274.SZ', '000615.SZ', '000638.SZ', '002532.SZ', '300099.SZ']


In [120]:
print(nextDayInfo)
if '601818.SH' in nextDayInfo.ts_code:
    print('success')

ts_code      000159.SZ
close            10.88
pre_close        11.61
Name: 000159.SZ, dtype: object


In [83]:
import pymysql
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='123456',
                             db='mybatis')
cursor = connection.cursor()
sql = "SELECT * FROM users WHERE id < 4"
try:
    cursor.execute(sql)
    results = cursor.fetchall()
    for row in results:
        print(row)
except:
    print ("Error: unable to fetch data")
connection.close()

(1, 'flee', 27)
(2, 'BillyBeauty', 18)
(3, 'jerrery', 30)


In [99]:
from pyecharts import Bar

attr = ["2013", "2014", "2015", "2016", "2017", "2018"]
v1 = [5, 20, 36, 10, 75, 90]
v2 = [10, 25, 8, 60, 20, 80]
v3 = [10, 25, 8, 60, 20, 80]
bar = Bar("涨停隔日分布图")
bar.add('[-10%, -7%]', attr, v1, is_stack=True, legend_top = 'bottom')
bar.add("(-7%, -3%]", attr, v2, is_stack=True, legend_top = 'bottom')
bar.add("(-3%, 0%)", attr, v1, is_stack=True, legend_top = 'bottom')
bar.add("[0%, 3%)", attr, v1, is_stack=True, legend_top = 'bottom')
bar.add("[3%, 7%)", attr, v2, is_stack=True, legend_top = 'bottom')
bar.add("[7%, 10%]", attr, v2, is_stack=True, legend_top = 'bottom')
bar.add("连板", attr, v2, is_stack=True, legend_top = 'bottom')
bar

In [25]:

import numpy as np
from pandas import DataFrame
# DataFrame(data,columns = ['col1','col0','col2'])
df = DataFrame(np.array([[1,2],[3,4]]), columns=['col1','col2'], index=['a','b'])
df2 = DataFrame([], columns=['code','name'])


In [26]:
print(df2)

Empty DataFrame
Columns: [code, name]
Index: []


In [1]:
##制作每日追涨报表
import requests
import time
import tushare as ts
import numpy as np
import os
import traceback
import decimal 

########################################

###挖地兔用户token，该token因积分较低，无法获取部分接口数据，需替换成自己账号的token
###挖地兔注册链接 https://tushare.pro/register?reg=248534

generateExcelFlag = True  ###是否生成表格开关
token = 'db8fbd4b0b8ec4e4f012937c9e1c3a36d5d1b1c6a293eba2d40a4fa3' 

easyMoneyUrl_A = 'http://nuff.eastmoney.com/EM_Finance2015TradeInterface/JS.ashx'
easyMoneyUrl_B = 'http://guba.eastmoney.com/list,'
flashDataUrl = 'http://home.flashdata2.jrj.com.cn/limitStatistic/zt/'
flushUrl_A = 'http://stockpage.10jqka.com.cn/'
head = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36'}


excelColumnNum = 12  ###表格列数
peroid = 30          ###交易数据分析周期
duration = 3         ###成交量获取天数               
                                        
currentDate = time.strftime('%Y%m%d',time.localtime(time.time()))
currentDateWithTime = time.strftime('%Y-%m-%d %H点%M分',time.localtime(time.time()))
currentTimeStamp = int(time.time()*1000)

###如果配置文件存在，则获取本地配置的token和配置的秘密网址
try:
    settingFile = open('/data/stock-setting.txt', 'r')
    settings = settingFile.read()
    token = settings.split()[0]
    secretUrl = settings.split()[1]
    settingFile.close()
    
    ###获取秘密网站数据
    response = requests.get(secretUrl)
    text = response.text
    textArray = text.split('\n')
    stockNum = 30             ###股票数量
    stockCodes = []
    for i in range(1, stockNum+1):
        stockText = textArray[i]
        stockCode = stockText.split(',')[1]
        stockCodes.append(str(stockCode))
except IOError:
    print('File is not accessible.')
    stockCodes = ['000159', '600345', '300663', '600831', '000990', '002307', '300536', '300171', '000993', '002473', '000788', '600232', '603169', '000905', '000610', '600722', '600072', '600800', '000156', '600199']
    stockNum = len(stockCodes)  
    
pro = ts.pro_api(token)
################## 函数 ####################

def getCodeWithSuffix(code):
    if code[0] == '6':
        return code + '.SH'
    else: 
        return code + '.SZ'
    
def formatVolume(var):
    if (var < 10000):
        return round(var/10000, 1)
    else: 
        return int(round(var/10000))

def calculateLimitPrice(price):
    lowRate = 0.9
    highRate = 1.1
    smallNum = 0.00001
    low = round(price*lowRate + smallNum, 2)
    high = round(price*highRate + smallNum, 2)
    return [low, high]

def isLimitHighPrice(close, pre_close):
    limitHighPrice = calculateLimitPrice(pre_close)
    if limitHighPrice[1] == close:
        return 1
    else:
        return 0
    
def getCodeWithNumSuffix(code):
    if code[0] == '6':
        return code + '1'
    else: 
        return code + '2'
    
def constructUrl_A(code, url):
    code = getCodeWithNumSuffix(code)
    return url + '?id=' + code

def constructUrl_B(code, url):
    return url + code + '.html'


def removeQuotationMarks(var):
    return var.split("\"",)[1]


    ###当卖单全空展示买一量，否则展示-1*卖一量
def finalOrder(code):
    finalOrderUrl = constructUrl_A(code, easyMoneyUrl_A)
    response = requests.get(finalOrderUrl)
    text = response.text  
    priceAmount = text.split(',')[4:24]
    for i in range(0, len(priceAmount)):
        tmp = priceAmount[i]
        priceAmount[i] = removeQuotationMarks(tmp)
    if priceAmount[5] == '-':
        return priceAmount[10]
    else:
        return '-' + priceAmount[15]      
    
def getDailyViewRank(code):
    finalGuBaUrl = constructUrl_B(code, easyMoneyUrl_B)
    response = requests.get(finalGuBaUrl)
    tmp1 = response.text.split('<em class="ran_num">')
    if len(tmp1) == 1:
        return 'not found'
    else:
        tmp2 = tmp1[1].split('</em> 名</span>')
        return tmp2[0]
    
def getLimitUpStockInfo(date, url):
    flashDataUrl = url + date + '.js'
    response = requests.get(flashDataUrl)
    (a,b,c) = response.text.partition(",\"Data\":[[")
    (stockInfoStr,e,f) = c.partition("]]};")
    tmpStockInfo = stockInfoStr.split("],[")
    stockInfo = []
    for i in range(0, len(tmpStockInfo)):  
        tmp = []
        singleStock = tmpStockInfo[i].split(',')
        stockInfo.append(singleStock) 
    return stockInfo

def mkdir(path):
    path = path.strip()
    path = path.rstrip("\\")
    isExists = os.path.exists(path)
    if not isExists:
        os.makedirs(path) 
        print(path +' 创建成功')
        return True
    else:
        print(path +' 目录已存在')
        return False
                     
###获取交易日历
calendar = pro.trade_cal(is_open = 1, end_date = currentDate, fields = 'cal_date')
calendarCut = calendar[-peroid:]
startDate = calendarCut.iloc[0, 0]
endDate = calendarCut.iloc[peroid-1, 0]

###当日涨停股记录
limitUPMap = {}
stockInfo = getLimitUpStockInfo(endDate, flashDataUrl)
for i in range(0, len(stockInfo)):
    code = removeQuotationMarks(stockInfo[i][0])
    limitUpTime = removeQuotationMarks(stockInfo[i][2])
    limitUPMap[code] = limitUpTime

###获取交易数据
result = []
for i in range(0, len(stockCodes)): 
    try:
        rowResult = [0]*excelColumnNum
        code = stockCodes[i]
        codeWithSuffix = getCodeWithSuffix(code)
        
        ###获取股票名称
        nameHistory = pro.namechange(ts_code = codeWithSuffix, fields='name')
        name = nameHistory.iloc[0, 0]
        
        
        ###获取成交量
        recentTrade = pro.daily(ts_code = codeWithSuffix, start_date=startDate, end_date=endDate, fields = 'trade_date, close, pre_close, vol')
        recentVolume = ''
        for j in range(0, duration):
            if j == 0:
                recentVolume =  str(formatVolume(recentTrade.iloc[j, 3]))
            elif j < recentTrade.shape[0]:  ###考虑复牌情况
                recentVolume = str(formatVolume(recentTrade.iloc[j, 3])) + '-' + recentVolume  
        recentVolume = '[' + recentVolume + ']'
 
            
        ###获取东财当日浏览排名
        viewRank = getDailyViewRank(code)
        
        ###封板时间
        limitUpTime = '-'
        if code in limitUPMap:
            limitUpTime = limitUPMap[code]
            
        ###获取换手率、总市值、股票单价  
        dailyInfo = pro.daily_basic(ts_code = codeWithSuffix, trade_date = endDate, fields = 'turnover_rate_f, circ_mv, close')
        #print(dailyInfo)
        closePrice = dailyInfo.iloc[0, 0]
        turnoverRate = str(round(dailyInfo.iloc[0, 1], 2)) + '%'
        circ_mv = str(round(dailyInfo.iloc[0, 2]/10000, 2)) + '亿'  
            
        ###获取连板数
        limitCount = peroid
        for j in range(0, len(recentTrade)):
            if isLimitHighPrice(recentTrade.iloc[j, 1], recentTrade.iloc[j, 2]) == 0:
                limitCount = j
                if j > 0 and limitUpTime == '-':
                    limitUpTime = '一字板'
                break
    
        ###获取封单数 当股票收盘未涨停时返回空
        orderNum = '-'
        if limitCount > 0:
            orderNum = finalOrder(code)     
        
        ###获取股票题材
        theme = '-'
        response = requests.get(flushUrl_A + code + '/', headers = head)
        text = response.text
        tmp1 = text.split('<dt>涉及概念：</dt>\n\t\t\t\t\t<dd title=\"')
        if len(tmp1) > 1:   
            tmp2 = tmp1[1].split('\">')
            theme = tmp2[0]
            
        ###数据转配
        rowResult[0] = i+1
        rowResult[1] = str(code)
        rowResult[2] = name
        rowResult[3] = viewRank
        rowResult[4] = recentVolume
        rowResult[5] = turnoverRate
        rowResult[6] = closePrice
        rowResult[7] = circ_mv
        rowResult[8] = limitCount
        rowResult[9] = limitUpTime
        rowResult[10] = orderNum
        rowResult[11] = theme
        result.append(rowResult)
        
    except BaseException as e:
        print('处理code=' + code + '时出现异常', e)
        traceback.print_exc()
             
print(result)

if generateExcelFlag:
    directory = '/data/stock/'
    mkdir(directory)
    fileName = '[BestStock]' + str(currentDateWithTime) + '.xls'
    columnName = '编号\t股票代码\t股票名称\t热度排名\t三日成交量(万手)\t换手率\t收盘价\t流通市值\t连板数\t封板时间\t封板单\t题材\n'
    output = open(directory + fileName,'w',encoding='gbk')
    output.write(columnName)
    for i in range(len(result)):
        for j in range(len(result[i])):
            output.write(str(result[i][j]))
            output.write('\t')  
        output.write('\n')       
    output.close()

[[1, '300220', '金运激光', '679', '[5-9-15]', '24.66%', 23.56, '29.43亿', 4, '14:16:09', '22782', '3D打印'], [2, '603360', '百傲化学', '630', '[12-14-21]', '28.24%', 30.28, '23.74亿', 4, '14:33:25', '46208', ''], [3, '600278', '东方创业', '95', '[8-51-56]', '33.13%', 18.77, '98.02亿', 5, '10:40:02', '181275', '科创板IPO，创投，自由贸易港，参股券商，上海国资改革，物流电商平台，上海自贸区，迪士尼'], [4, '000590', '启迪古汉', '236', '[0.4-0.3-0.9]', '0.78%', 15.65, '34.94亿', 6, '一字板', '143961', '雄安新区，高校，健康中国'], [5, '002835', '同为股份', '498', '[13-29-28]', '44.12%', 12.24, '10.34亿', 3, '14:50:45', '12716', '安防'], [6, '600279', '重庆港九', '144', '[3-3-37]', '8.98%', 6.13, '42.48亿', 3, '一字板', '131578', '白马股，一带一路'], [7, '002265', '西仪股份', '177', '[2-48-51]', '42.01%', 13.72, '41.72亿', 1, '14:51:48', '23751', '证金持股，军工'], [8, '601008', '连云港', '39', '[109-149-211]', '40.38%', 6.24, '63.35亿', 0, '-', '-', '中韩自贸区，一带一路'], [9, '600146', '商赢环球', '327', '[43-53-58]', '30.94%', 12.41, '24.82亿', 3, '13:50:14', '31599', '商誉减值，跨境电商，区块链，互联网金融，上海自贸区，电子商务，碳纤维，融资融券'], [10, '0

NameError: name 'result' is not defined